In [2]:
import Pkg
Pkg.activate(@__DIR__)
# Pkg.instantiate()
import MathOptInterface as MOI
import Ipopt
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx
import ECOS
using LinearAlgebra
using Plots
using Random
using JLD2
using Test
using MeshCat
const mc = MeshCat
using StaticArrays
using Printf

include(joinpath(@__DIR__, "utils", "fmincon.jl"))
include(joinpath(@__DIR__, "utils", "quaternions.jl"))
include(joinpath(@__DIR__, "utils", "quadrotor.jl"))
include(joinpath(@__DIR__, "utils", "util.jl"))

  Activating environment at `~/Desktop/16745/recitations-2024/16745proj/Project.toml`


create_idx (generic function with 1 method)

In [3]:
function quad_costs(n_lift, m_lift, scenario=:p2pa)
    if scenario == :hover
        q_diag = 10.0 * ones(n_lift)
        q_diag[4:7] .= 1e-6

        r_diag = 1.0e-3 * ones(m_lift)
        r_diag[end] = 1

        qf_diag = copy(q_diag) * 10.0
    elseif scenario == :p2pa
        q_diag = 1.0 * ones(n_lift)
        q_diag[1] = 1e-5
        # q_diag[4:7] .*= 25.0
        # q_diag

        r_diag = 1.0e-3 * ones(m_lift)
        # r_diag = 1.0e-3*ones(m_lift)
        r_diag[end] = 1

        qf_diag = 100 * ones(n_lift)
    else
        q_diag = 1e-1 * ones(n_lift)
        q_diag[1] = 1e-3
        q_diag[4:7] .*= 25.0

        r_diag = 2.0e-3 * ones(m_lift)

        # r_diag = 1.0e-3*ones(m_lift)
        r_diag[end] = 1

        qf_diag = 100 * ones(n_lift)
    end
    return q_diag, r_diag, qf_diag
end

function load_costs(n_load, m_load, scenario=:p2pa)
    if scenario == :hover
        q_diag = 10.0 * ones(n_load) #

        r_diag = 1 * ones(m_load)
        qf_diag = 10.0 * ones(n_load)
    elseif scenario == :p2p
        q_diag = 1.0 * ones(n_load) #

        # q_diag = 0*ones(n_load)
        q_diag[1] = 1.0e-5
        r_diag = 1 * ones(m_load)
        qf_diag = 0.0 * ones(n_load)

        # q_diag = 1000.0*ones(n_load)
        # r_diag = 1*ones(m_load)
        # qf_diag = 1000.0*ones(n_load)
    else
        q_diag = 0.5e-1 * ones(n_load) #

        # q_diag = 0*ones(n_load)
        # q_diag[1] = 1e-3
        r_diag = 1 * ones(m_load)
        qf_diag = 0.0 * ones(n_load)

        # q_diag = 1000.0*ones(n_load)
        # r_diag = 1*ones(m_load)
        # qf_diag = 1000.0*ones(n_load)

    end
    return q_diag, r_diag, qf_diag
end


load_costs (generic function with 2 methods)

In [4]:
function quad_cost(params::NamedTuple, Z::Vector)::Real
    idx, N = params.idx, params.N
    Q, R, Qf = params.q_load, params.r_load, params.qf_load

    #xg should be a stacked vector of goals [x1g;x2g;x3g...]
    xg = params.xg


    #LQR cost 
    J = 0
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]

        J += 0.5 * (xi - xg)' * Q * (xi - xg)
        J += 0.5 * ui' * R * ui

    end

    # dont forget terminal cost 
    J += 0.5 * (Z[idx.x[N]] - xg)' * Q * (Z[idx.x[N]] - xg)

    return J
end

# #To add to params/model:
# n_lift = 13
# m_lift = 6 #5 + 1 for quaternion slack variable 
# n_load = 6
# m_load = NUM_QUAD # num quad /L

quad_cost (generic function with 1 method)

In [10]:
# q_lift, r_lift, qf_lift = quad_costs(n_lift, m_lift, scenario)

# cat([q_lift for _ in 1:n]...; dims=(1,2))
# q_load, r_load, qf_load = load_costs(n_load, m_load, scenario)

function system_cost(params::NamedTuple, Z::Vector)::Real
    # cost function 
    idx, N, xg = params.idx, params.N, params.xg
    Q, R, Qf = params.Q, params.R, params.Qf
    Xref, Uref = params.Xref, params.Uref

    J = 0
    for i = 1:N-1
        J += 0.5 * (Z[idx.x[i]] - Xref[i])' * Q * (Z[idx.x[i]] - Xref[i]) +
             0.5 * (Z[idx.u[i]] - Uref[i])' * R * (Z[idx.u[i]] - Uref[i])
    end
    J += 0.5 * (Z[idx.x[N]] - Xref[N])' * Qf * (Z[idx.x[N]] - Xref[N])
    return J
end

function dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    M1, M2 = params.M1, params.M2
    J1, J2 = params.J1, params.J2
    model = params.model

    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)

    for i = 1:N-1
        #3
        xk = Z[idx.x[i]]
        uk = Z[idx.u[i]]
        c[idx.c[i]] = Z[idx.x[i+1]] - rk4(model, combined_system_dynamics, xk, uk, dt)
    end

    return c
end

function quaternion_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    model = params.model

    # create c -1in a ForwardDiff friendly way (check HW0)
    # c = Vector{eltype    c = zeros(eltype(Z), 1)
    c = zeros(eltype(Z), 1)
    for i = 1:N-1
        x = Z[idx.x[i]]
        u = Z[idx.u[i]]
        for k = 1:NUM_QUAD
            quat = x[13*(k-1)+4:13*(k-1)+8]
            s = u[6*NUM_QUAD]
            push!(c, norm(s * quat, 2) - 1)
        end
    end
    return c
end

function equal_tension_cable_constraint(params::NamedTuple, Z::Vector)::Vector
    idx, N = params.idx, params.N
    m_lift = params.m_lift
    NUM_QUAD = params.NUM_QUAD
    # c = Vector{eltype(Z)}()
    c = zeros(eltype(Z), 1)
    for i = 1:N-1
        ui = Z[idx.u[i]]
        s_inds = m_lift:m_lift:m_lift*NUM_QUAD
        ui_load = ui[(1:NUM_QUAD).+m_lift*NUM_QUAD]
        for i = 1:NUM_QUAD
            # println(ui[s_inds[i]])
            push!(c, ui[s_inds[i]] - ui_load[i])
        end
    end
    return c
end

function rope_length_constraint(params::NamedTuple, Z::Vector)::Vector
    idx, N = params.idx, params.N
    # c = Vector{eltype(Z)}()
    c = zeros(eltype(Z), 1)
    for i = 1:N-1
        xi = Z[idx.x[i]]
        r_load = xi[(1:3).+n_lift*NUM_QUAD]
        xi_quads = [xi[(quad-1)*n_lift+1:n_lift*quad] for quad = 1:NUM_QUAD]
        for j = 1:NUM_QUAD
            push!(c, norm(xi_quads[j][1:3] - r_load)^2)
        end
    end
    return c
end

function equality_constraints(params::NamedTuple, Z::Vector)::Vector
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg
    return [Z[idx.x[1]] - xic;
        Z[idx.x[N]] - xg
    dynamics_constraints(params, Z)]
    quaternion_constraints(params, Z);
    # equal_tension_cable_constraint(params, Z);
    # rope_length_constraint(params, Z)]
end

equality_constraints (generic function with 1 method)

In [6]:
function proximity_constraint(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    n_lift = params.n_lift
    NUM_QUAD = params.NUM_QUAD
    dquad = params.dquad
    #Assuming obstacle positions is of form [[x1,y1],[x2,y2]]
    # c = Vector{eltype(Z)}()
    c = zeros(eltype(Z), 1)
    for i = 1:N-1 #Each timestep
        xi = Z[idx.x[i]]
        xi_quads = [xi[(quad-1)*n_lift+1:n_lift*quad] for quad = 1:NUM_QUAD]
        for j = 1:NUM_QUAD
            for k = j+1:NUM_QUAD
                constraint_val = 2 * dquad - norm(xi_quads[j][1:2] - xi_quads[k][1:2])^2
                push!(c, constraint_val)
            end
        end
    end
    return c
end

function obstacle_proximity_constraint(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    NUM_QUAD = params.NUM_QUAD
    n_lift = params.n_lift
    obstacle_positions = params.obstacles
    dquad, dobs = params.dquad, params.dobs
    #Assuming obstacle positions is of form [[x1,y1],[x2,y2]]
    c = zeros(eltype(Z), 1)
    for i = 1:N-1 #Each timestep
        xi = Z[idx.x[i]]
        xi_quads = [xi[(quad-1)*n_lift+1:n_lift*quad] for quad = 1:NUM_QUAD]
        for j = 1:length(obstacle_positions) #each obstacle
            for k = 1:NUM_QUAD
                constraint_val = dquad + dobs - norm(xi_quads[k][1:2] - obstacle_positions[j])^2
                push!(c, constraint_val)
            end
        end
    end
    return c
end

function load_proximity_constraint(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    NUM_QUAD = params.NUM_QUAD
    n_lift = params.n_lift
    obstacle_positions = params.obstacles
    dload, dobs = params.dload, params.dobs
    #Assuming obstacle positions is of form [[x1,y1],[x2,y2]]
    c = zeros(eltype(Z), 1)
    for i = 1:N-1 #Each timestep
        xi = Z[idx.x[i]]
        xi_load = xi[1:2 .+ n_lift*NUM_QUAD]
        for j = 1:length(obstacle_positions) #each obstacle
            constraint_val = dload + dobs - norm(xi_load - obstacle_positions[j])^2
            push!(c, constraint_val)
        end
    end
    return c
end

function positive_cable_tension(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    m_lift = params.m_lift
    NUM_QUAD = params.NUM_QUAD
    c = zeros(eltype(Z), 1)
    for i = 1:N-1 #Each timestep
        ui = Z[idx.u[i]]
        ui_load = ui[(1:NUM_QUAD).+m_lift*NUM_QUAD]
        for j = 1:length(ui_load)
            push!(c, -ui_load[j])
        end
    end
    return c
end

function inequality_constraints(params::NamedTuple, Z)
    return []
    # return [proximity_constraint(params, Z);
    #     obstacle_proximity_constraint(params, Z);
    #     load_proximity_constraint(params, Z);
    #     positive_cable_tension(params, Z)]
end

inequality_constraints (generic function with 1 method)

In [7]:
function states_from_center(NUM_QUAD, load_pos, r=3)
    x = [zeros(13) for i = 1:NUM_QUAD+1]
    q10 = [0.99115, 4.90375e-16, 0.132909, -9.56456e-17]
    for i = 1:NUM_QUAD
        x[i] = [r * cos(i * 2 * pi / N); r * sin(i * 2pi / N); 1; q10; zeros(13 - 7)] + load_pos
    end
    #Add Load State
    x[end] = [load_pos[1:3]; zeros(3)]
    return x
end


function reference_trajectory(N, xic, xg)
    return collect(range(vcat(xic...), vcat(xg...), length=N))
end

# xic = states_from_center(NUM_QUAD, [-1.5; -1.5; 1.5; zeros(13-3)])
# xg = states_from_center(NUM_QUAD, [1.5; 1.5; 1.5; zeros(13-3)])

# x = reference_trajectory(4, xic, xg)
# println(size(x[1]))

reference_trajectory (generic function with 1 method)

In [11]:
#--------PLACEHOLDER
tf = 10
dt = 0.1
t_vec = 0:dt:tf
N = length(t_vec)
#-----------------
# problem size 
L = num_lift = NUM_QUAD = 4
nx = 13 * L + 6
nu = 6 * L + L # 5 true controls + 'slack' variable to normalize quat

# create indexing utilities 
idx = create_idx(nx, nu, N)


# index sets 
M1 = vcat([(i - 1) * 10 .+ (1:5) for i = 1:5]...)
M2 = vcat([((i - 1) * 10 + 5) .+ (1:5) for i = 1:4]...)
J1 = [5, 15, 25, 35]
J2 = [10, 20, 30, 40]

# dynamics parameters 
model = (lift=quad_params, load=load_params)

#Initial Load and Goal Load
xic_load = [-1.5; -1.5; 1.5]
xg_load = [1.5; 1.5; 1.5]

# initial and goal states 
xic = states_from_center(NUM_QUAD, [xic_load; zeros(13 - 3)])
xg = states_from_center(NUM_QUAD, [xg_load; zeros(13 - 3)])

# reference trajectory 
Xref = reference_trajectory(N, xic, xg)
Uref = [0.01 * randn(idx.nu) for i = 1:N-1]

n_lift = 13
m_lift = 6 #5 + 1 for quaternion slack variable 
n_load = 6
m_load = NUM_QUAD
# LQR cost function (tracking Xref, Uref)
q_lift, r_lift, qf_lift = quad_costs(n_lift, m_lift)
q_load, r_load, qf_load = load_costs(n_load, m_load)
Q = diagm(vcat([q_lift for _ in 1:NUM_QUAD]..., q_load))
R = diagm(vcat([r_lift for _ in 1:NUM_QUAD]..., r_load))
Qf = diagm(vcat([qf_lift for _ in 1:NUM_QUAD]..., qf_load))

# put everything useful in params 
params = (
    model=model,
    nx=nx,
    nu=nu,
    tf=tf,
    dt=dt,
    t_vec=t_vec,
    N=N,
    M1=M1,
    M2=M2,
    J1=J1,
    J2=J2,
    xic=vcat(xic...),
    xg=vcat(xic...),
    idx=idx,
    Q=Q, R=R, Qf=Qf,
    Xref=Xref,
    Uref=Uref,
    NUM_QUAD=NUM_QUAD,
    n_lift=n_lift,
    m_lift=m_lift, #5 + 1 for quaternion slack variable 
    n_load=n_load,
    m_load=NUM_QUAD, # num quad /L    dcable = 1.55,
    obstacles=[], #Set THESE
    dload=1.55,
    dquad=0.5,
    dobs=1,
)





# TODO: primal bounds
x_l = -Inf * ones(idx.nz) # update this 
x_u = Inf * ones(idx.nz) # update this 
# umax = 5 * ones(4)
# rmin = [-Inf -Inf 1]
# rmax = [Inf Inf 5]
# for i = 1:N-1
#     #Control Bounds
#     u_inds = [(1:4) .+ i for i in (0:m_lift:m_lift*NUM_QUAD-1)]
#     for u_index = u_inds
#         x_l[idx.u[i]][u_index] .= 0
#         x_u[idx.u[i]][u_index] .= umax
#     end
#     #Position Bounds
#     r_inds = [(1:3) .+ i for i in (0:params.n_lift:params.n_lift*NUM_QUAD)]
#     for x_idx = r_inds
#         x_l[idx.x[i]][x_idx] = rmin
#         x_u[idx.x[i]][x_idx] = rmax
#     end
# end


# TODO: inequality constraint bounds
c_l = -Inf * ones(0) # update this 
c_u = zeros(0) # update this 

# TODO: initialize z0 with the reference Xref, Uref 
z0 = zeros(idx.nz) # update this
for i = 1:N
    z0[idx.x[i]] = Xref[i]
end
for i = 1:N-1
    z0[idx.u[i]] = Uref[i]
end
# adding a little noise to the initial guess is a good idea 
z0 = z0 + (1e-6) * randn(idx.nz)

diff_type = :auto

# @show size(xic)
# equal_tension_cable_constraint(params, z0)

# @show size(c_l)
# @show size(c_u)
# @show size(inequality_constraints(params, z0))
Z = fmincon(system_cost, equality_constraints, inequality_constraints,
    x_l, x_u, c_l, c_u, z0, params, diff_type;
    tol=1e-6, c_tol=1e-6, max_iters=100, verbose=true)

# # pull the X and U solutions out of Z 
# X = [Z[idx.x[i]] for i = 1:N]
# U = [Z[idx.u[i]] for i = 1:(N-1)]


---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
